<a href="https://colab.research.google.com/github/GilliesK/Deleuze-bank/blob/master/Mask%20and%20Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TASKS:
# 1.	Take a team video
# 2.	Segment out the people
# 3.	Take Linda’s or Ning’s videos and segment a cell over time
# 4.	Then interpose your image or video inside the cell
# 5.	Take one person and have them walk around the cell


In [ ]:
%%capture
# Loading libraries
import random                        # Library to generate random numbers
import skimage                       # Library for image manipulation
import numpy as np                   # Library for array manipulation
import urllib.request                # Library to download data
import matplotlib.pyplot as plt      # Library used for plotting
from skimage import io               # Module from skimage
from skimage.io import imread        # Module from skimage to read images as numpy arrays
from skimage.filters import gaussian # Module working with a gaussian filter
from skimage import measure
from scipy import ndimage as ndi              # Distance Transform
from skimage.feature import peak_local_max    # Local maxima in a matrix
from skimage.segmentation import watershed    # Watershed algorithm
from skimage.draw import polygon

from skimage.morphology import binary_dilation, watershed
from skimage import measure
from skimage.morphology import binary_dilation, watershed
from skimage.draw import polygon

%matplotlib inline

%matplotlib inline

%cd gdrive/MyDrive/
! pip install scikit-video

import skvideo.io

# New Section

In [ ]:
fName = './segmentation_video.mp4'  
vid = skvideo.io.vread(fName)  

ALL_LABELS=[]

for t in range(vid.shape[0]):
  img = vid[t,:,:,0]

  # Thresholding the image
  threshold = 90   # Please play  with this threshold
  mask_image = np.zeros(img.shape)
  mask_image[img>threshold] = 255

  # Applying a gaussian filter to the image
  new_mask = gaussian(mask_image, sigma=12) 

  contours = measure.find_contours(new_mask, level=250) # level is half of 255 (ish). What happens if we change it?
  
  #contours_input = 
  contours_connected = np.vstack((sorted(contours, key=len, reverse=True)[0:3]))

  watershed_starting_mask = np.zeros(img.shape).astype(int)                    # Prealocating an array with zeros. Notice the datatype.
  rr, cc = polygon(contours_connected[:,0], contours_connected[:,1])   # Returns the coordinates inside the contour
  watershed_starting_mask[rr,cc] = 1      
  
    # Apply watershed
  distance = ndi.distance_transform_edt(watershed_starting_mask)                       # Computes the Distance Transform distance in the image
  coords = peak_local_max(distance, min_distance=120, labels=watershed_starting_mask)   # Use the Distance transform image to find local maxima
  _,inds = np.unique(distance[coords[:,0],coords[:,1]],return_index=True)      # Make sure they are unique
  coords = coords[inds,:]                                                      # Selecting unique indexes
  mask = np.zeros(distance.shape, dtype=bool)                                  # Prealocating an array with zeros
  mask[tuple(coords.T)] = True                                                 # Make an image with 1's where local maxima are
  markers, _ = ndi.label(mask)                                                 # Unique values used as the desired labels

  # Using the watershed algorithm
  labels = watershed(-distance, markers, mask=watershed_starting_mask, watershed_line=True)                                      # Replacing all values inside the contour with ones.

  ALL_LABELS.append(labels)

    # Plotting the results

  if t==0:
    IMG=img
    WSM=watershed_starting_mask
    third=ndi.distance_transform_edt(watershed_starting_mask)
    four_x=coords[:,1]
    four_y=coords[:,0]
    LABELS=labels


FileNotFoundError: ignored

In [ ]:
    #FOR t=0
    
    f,ax = plt.subplots(1,5, figsize=(15,7))
    ax[0].imshow(IMG, cmap='Spectral')
    ax[0].set_title('origninal')
    ax[1].imshow(WSM, cmap='Greys_r')
    ax[1].set_title('Mask')
    ax[2].imshow(third, cmap='Greys')
    ax[2].set_title('Distance Transform')
    ax[3].imshow(third, cmap='Greys')
    ax[3].scatter(four_x,four_y,c='r')
    ax[3].set_title('Local Maxima in Dist. Transform')
    ax[4].imshow(LABELS, cmap='Spectral')
    ax[4].set_title('Masks with Labels')
    f.tight_layout() 